In [ ]:
%pip install torch transformers peft

In [2]:
import torch
# Проверяем доступность GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from transformers import BertTokenizerFast, AutoModelForSequenceClassification
from peft import PeftModel

base_model_name = "blanchefort/rubert-base-cased-sentiment-rusentiment"
tokenizer = BertTokenizerFast.from_pretrained(base_model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
# Перемещение модели на устройство
base_model.to(device)

In [5]:
def predict(text, model, tokenizer):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted

def number_to_string(value):
  mapping = {
      0: "NEUTRAL",
      1: "POSITIVE",
      2: "NEGATIVE"
  }
  return mapping.get(value, "Invalid number")

def get_model_class_answer(text: str, model, tokenizer) -> str:
  predicted_class_label = predict(text, model, tokenizer)[0]
  return number_to_string(predicted_class_label)

In [10]:
staying_possibility_adapter_name = "Vzvorygin/sirius_hack_staying_possibility"
staying_possibility_model = PeftModel.from_pretrained(base_model, staying_possibility_adapter_name)

adapter_config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

In [32]:
get_model_class_answer('Я нахожусь в процессе поиска информации и анализа своих возможностей', staying_possibility_model, tokenizer)

'NEUTRAL'

In [33]:
recommendation_possibility_adapter_name = "Vzvorygin/sirius_hack_recommendation_possibility"
recommendation_possibility_model = PeftModel.from_pretrained(base_model, recommendation_possibility_adapter_name)

In [34]:
get_model_class_answer('Я нахожусь в процессе поиска информации и анализа своих возможностей', recommendation_possibility_model, tokenizer)

'POSITIVE'

In [35]:
returning_possibility_adapter_name = "Vzvorygin/sirius_hack_returning_possibility"
returning_possibility_model = PeftModel.from_pretrained(base_model, returning_possibility_adapter_name)

In [36]:
get_model_class_answer('Я нахожусь в процессе поиска информации и анализа своих возможностей', returning_possibility_model, tokenizer)

'NEUTRAL'

In [37]:
get_model_class_answer('Все зависит от зарплаты.', base_model, tokenizer)

'NEUTRAL'